Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
!pip install yfinance
!pip install pandas-datareader

In [2]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [3]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = "IBM"

In [4]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = yf.download(stock_symbol, "2017-12-29", "2021-1-1")
df = df[["Open", "High", "Low", "Close"]]
df = df.dropna()

[*********************100%***********************]  1 of 1 completed


In [5]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df["Close"].rolling(window=15).mean() # Average last 15 days
                    # = df["Close"] - df["Close"].shift(1)
df['Open_1_Change'] = df["Open"].diff(1) # Today - Yesterday
df = df.dropna()
X = df.copy()
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2018-01-19,157.227539,158.126190,154.206497,155.229446,154.697261,-5.296356
2018-01-22,154.407272,156.768646,154.024857,155.449326,155.242830,-2.820267
2018-01-23,155.659653,158.948380,155.363297,158.938812,156.060549,1.252380
2018-01-24,159.416824,161.300186,157.571701,158.097519,156.769281,3.757172
2018-01-25,158.613770,159.445511,157.925430,158.193115,157.214150,-0.803055
...,...,...,...,...,...,...
2020-12-24,119.502869,119.598473,118.747612,119.206497,119.543020,1.070747
2020-12-28,119.598473,121.032501,118.986618,119.330788,119.391331,0.095604
2020-12-29,119.837479,119.961761,117.820267,118.355644,119.333970,0.239006


In [6]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.
            # Shift, rolling, diff, 
            # + => pervious values
            # - => future values
            # Close Price Only
            # Tomorrow -  Today
            # Tomorrow > Today =   1
            # Tomorrow <= Today = -1
            # A > B
            # A - B > 0
y = np.where(df["Close"].shift(-1) > df["Close"], 1, -1)
y

array([ 1,  1, -1,  1,  1, -1, -1,  1, -1, -1, -1,  1, -1, -1,  1,  1, -1,
        1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1,  1,  1, -1,  1, -1,  1,
        1, -1, -1,  1,  1, -1, -1,  1, -1, -1,  1, -1,  1,  1, -1, -1,  1,
       -1, -1,  1,  1, -1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1, -1,
       -1,  1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1,  1, -1, -1,  1, -1,
       -1, -1, -1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,
       -1, -1, -1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,
        1, -1, -1, -1,  1,  1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1,
       -1,  1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1,
        1, -1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,
       -1, -1,  1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,
       -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,  1,  1, -1,  1,  1,  1, -1,
        1, -1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1,  1, -1,  1,  1, -1,
        1, -1,  1, -1,  1

In [7]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X)) # Split into 70% Training
X_train = X[0:index]
X_test = X[index:]
y_train = y[0:index]
y_test =  y[index:]

In [8]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

C:\Users\buhli\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_))) # beta

,0,1
0,Open,[0.22920341557849794]
1,High,[-0.16249512211291442]
2,Low,[-0.1237365042645492]
3,Close,[0.0676583754580711]
4,Close_15_Rolling,[-0.013775955832831978]
5,Open_1_Change,[-0.012211347002545838]


In [10]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [11]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_test, predictions)

array([[24, 87],
       [18, 95]], dtype=int64)

In [12]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test, y_test)

0.53125

In [13]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(LogisticRegression(), X, y, cv=5)

C:\Users\buhli\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\buhli\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [14]:
cross_val

array([0.51006711, 0.53020134, 0.56375839, 0.53691275, 0.54054054])

In [15]:
cross_val.mean()

0.5362960275711954